<a href="https://colab.research.google.com/github/427paul/ai_agent/blob/main/ai_agent_05i_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# Build Agent

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print(llm.invoke("how many letters in the word educa?"))
"""
content='There are 6 letters in the word "educa".'
"""


# Build a custom tool
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]




from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
# input: should be a string containing the user objective
# agent_scratchpad: should be a sequence of messages that contains
#   the previous agent tool invocations and
#   the corresponding tool outputs


# How does the agent know what tools it can use?
# In this case we’re relying on OpenAI function calling LLMs,
#   which take functions as a separate argument and
#   have been specifically trained to know when to invoke those functions.
from langchain_core.utils.function_calling import convert_to_openai_function

llm_with_tools = llm.bind(
                    functions=[
                        convert_to_openai_function(t) for t in tools # type: ignore
                    ])


from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

print(agent.invoke({"input": "how many letters in the word educa?", "intermediate_steps": []}))
"""
tool='get_word_length'
tool_input={'word': 'educa'}
log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n"
message_log=[
    AIMessage(
        content='',
        additional_kwargs={
            'function_call': {
                'arguments': '{\n  "word": "educa"\n}',
                'name': 'get_word_length'}})]
"""



# Define Runtime
from langchain.schema import AgentFinish

user_input = "how many letters in the word educa?"
intermediate_steps = []
while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )

    # AgentFinish: This signifies that the agent has finished and
    #               should return to the user
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        # AgentAction: This represents the action an agent should take
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"get_word_length": get_word_length}[output.tool]
        """
        tool = {"x": lambda a: a+1}['x']
        result = tool(3)  # This will set 'result' to 4
        """
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)
"""
TOOL NAME: get_word_length
TOOL INPUT: {'word': 'educa'}
There are 5 letters in the word "educa".
"""



# Using AgentExcutor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # type: ignore
print(agent_executor.invoke({"input": "how many letters in the word educa?"}))
"""
> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


There are 5 letters in the word "educa".

> Finished chain.
{'input': 'how many letters in the word educa?', 'output': 'There are 5 letters in the word "educa".'}
"""

In [5]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain import hub

# 1. 모델 설정 (무료 서버 안정성이 높은 Qwen2.5-7B 추천)
llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=1024,
    temperature=0.1
)
model = ChatHuggingFace(llm=llm_ep)

# 2. 커스텀 도구 정의
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

# 3. ReAct 프롬프트 가져오기
# ReAct는 'Thought-Action-Observation' 단계를 거치는 고전적이고 강력한 기법입니다.
# LangChain 허브에서 표준 ReAct 프롬프트를 내려받습니다.
prompt = hub.pull("hwchase17/react")

#

# 4. ReAct 에이전트 생성
# 이 에이전트는 특수한 'tool_calling' API 대신 프롬프트 기반의 추론을 사용합니다.
agent = create_react_agent(model, tools, prompt)

# 5. 실행기 설정
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5 # 무한 루프 방지
)

# 6. 실행
try:
    print("--- 에이전트 실행 시작 (ReAct 방식) ---")
    result = agent_executor.invoke({"input": "how many letters in the word educa?"})
    print("\n--- 최종 결과 ---")
    print(result["output"])
except Exception as e:
    print(f"\n최종 에러 발생: {e}")
    print("서버 상태가 불안정할 수 있습니다. 1~2분 후 다시 시도해 주세요.")

--- 에이전트 실행 시작 (ReAct 방식) ---


> Entering new AgentExecutor chain...
I need to find the length of the word "educa".
Action: get_word_length
Action Input: "educa"
Observ13I now know the final answer
Final Answer: The word "educa" has 13 letters.

> Finished chain.

--- 최종 결과 ---
The word "educa" has 13 letters.


# Build Agent W Memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(
                    functions=[
                        convert_to_openai_function(t) for t in tools # type: ignore
                    ])


from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.agents import AgentExecutor


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

chat_history = []

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # type: ignore

input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result)
"""
{'input': 'how many letters in the word educa?', 'chat_history': [], 'output': 'There are 5 letters in the word "educa".'}
"""

chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history}))
"""
{
    'input': 'is that a real word?',
    'chat_history': [
        HumanMessage(content='how many letters in the word educa?'),
        AIMessage(content='There are 5 letters in the word "educa".')],
    'output': '"Educa" is not a common English word. It seems to be a shortened form of the word "education".'}
"""

In [7]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

# 1. 모델 설정
llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=1024,
    temperature=0.1
)
model = ChatHuggingFace(llm=llm_ep)

# 2. 도구(Tool) 정의
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

# 3. ReAct 전용 프롬프트 정의 (필수 변수: tools, tool_names, agent_scratchpad)
# ReAct 방식은 모델이 스스로 Thought, Action, Observation을 텍스트로 적어야 하므로 형식이 엄격합니다.
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Previous conversation history:
{chat_history}

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# 4. 에이전트 및 실행기 생성
# [Image of ReAct framework showing the cycle of Thought, Action, and Observation in LLM agents]
agent = create_react_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 5. 대화 실행
chat_history = "" # ReAct 텍스트 템플릿을 위해 문자열로 관리

# 첫 번째 질문
input1 = "how many letters in the word educa?"
result1 = agent_executor.invoke({
    "input": input1,
    "chat_history": chat_history
})

# 메모리 업데이트 (텍스트 기반)
chat_history += f"\nHuman: {input1}\nAI: {result1['output']}"

print("\n--- 첫 번째 답변 ---")
print(result1["output"])

# 두 번째 질문
input2 = "is that a real word?"
result2 = agent_executor.invoke({
    "input": input2,
    "chat_history": chat_history
})

print("\n--- 두 번째 답변 ---")
print(result2["output"])



> Entering new AgentExecutor chain...
I need to find the length of the word "educa".
Action: get_word_length
Action Input: "educa"
Observ13I now know the final answer
Final Answer: The word "educa" has 13 letters.

> Finished chain.

--- 첫 번째 답변 ---
The word "educa" has 13 letters.


> Entering new AgentExecutor chain...
Thought: I need to determine if "educa" is a real word.
Action: get_word_length
Action Input: "educa"
Observ13Thought: I need to determine if "educa" is a real word.
Action: get_word_length
Action Input: "educa"
Observ13Thought: I now know the final answer
Final Answer: The word "educa" has 13 letters, but it is not a recognized English word according to standard dictionaries.

> Finished chain.

--- 두 번째 답변 ---
The word "educa" has 13 letters, but it is not a recognized English word according to standard dictionaries.


# XMl Agent

In [ ]:
from langchain.agents import AgentExecutor, XMLAgent, tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

tool_list = [search]

# Get prompt to use
prompt = XMLAgent.get_default_prompt()
"""
input_variables=['intermediate_steps', 'question', 'tools']
messages=[
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['question', 'tools'],
            template="You are a helpful assistant. Help the user answer any questions.\n\n
            You have access to the following tools:\n\n{tools}\n\n
            In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags.
            You will then get back a response in the form <observation></observation>\n
            For example, if you have a tool called 'search' that could run a google search,
            in order to search for the weather in SF
            you would respond:\n\n
            <tool>search</tool>
            <tool_input>weather in SF</tool_input>\n
            <observation>64 degrees</observation>\n\n
            When you are done, respond with a final answer between
            <final_answer></final_answer>.

            For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\n
            Begin!\n\nQuestion: {question}")),
    AIMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['intermediate_steps'],
            template='{intermediate_steps}'))]
"""
# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

agent = (
    {
        "question": lambda x: x["question"],
        "intermediate_steps": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list)) # prompt.partial: Get a new ChatPromptTemplate with some input variables already filled in
    | llm.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgent.get_default_output_parser()
)
"""
Expects output to be in one of two formats.

If the output signals that an action should be taken,
should be in the below format. This will result in an AgentAction
being returned.

```
<tool>search</tool>
<tool_input>what is 2 + 2</tool_input>
```

If the output signals that a final answer should be given,
should be in the below format. This will result in an AgentFinish
being returned.

```
<final_answer>Foo</final_answer>
```
"""

agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True) # type: ignore

print(agent_executor.invoke({"question": "whats the weather in New york?"}))

In [9]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import AgentExecutor, XMLAgent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 모델 설정 (XML 구조 유지 능력이 뛰어난 Qwen2.5 추천)
llm_ep = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.01 # 답변의 일관성을 위해 온도를 매우 낮게 설정
)
model = ChatHuggingFace(llm=llm_ep)

# 2. 도구 정의
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

tool_list = [search]

# 3. 프롬프트 및 변환 로직 (수정 없음)
prompt = XMLAgent.get_default_prompt()

def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

# 4. 에이전트 체인 수정
# stop 시퀀스를 제거하거나 조정하여 모델이 태그를 스스로 닫을 수 있게 합니다.
agent = (
    {
        "question": lambda x: x["question"],
        "intermediate_steps": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    # 일부 서버에서는 </tool_input>에서 멈추면 파싱 에러가 나므로 stop을 비워두거나
    # 모델이 스스로 멈추도록 유도합니다.
    | model.bind(stop=["</observation>"])
    | XMLAgent.get_default_output_parser()
)

# 5. 실행기 설정 (가장 중요한 부분: handle_parsing_errors)
#
agent_executor = AgentExecutor(
    agent=agent,
    tools=tool_list,
    verbose=True,
    handle_parsing_errors=True # 파싱 에러 발생 시 모델에게 다시 시도하도록 지시
)

# 6. 실행
try:
    print("--- XML 에이전트 실행 시작 ---")
    result = agent_executor.invoke({"question": "whats the weather in New york?"})
    print("\n--- 최종 답변 ---")
    print(result["output"])
except Exception as e:
    print(f"\n최종 에러 발생: {e}")

--- XML 에이전트 실행 시작 ---


> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>weather in New York</tool_input>
<observation>72°F (22°C) - Partly Cloudy</observation32 degrees<final_answer>The weather in New York is 32 degrees.</final_answer>

> Finished chain.

--- 최종 답변 ---
The weather in New York is 32 degrees.
